In [ ]:
import sqlite3

In [ ]:
dbpath="/home/sergei/.qif-transaction-generator/db.sqlite"
conn = sqlite3.connect(dbpath)
c = conn.cursor()

In [ ]:
c.execute("SELECT * FROM receipts WHERE purchase_date > '2022-07-01'")
data = c.fetchall()
# print(data)
for row in data:
    print(row)